In [1]:
import pandas as pd
import os

# Directory
image_directory = "/kaggle/input/plant-disease-classification-merged-dataset"

# Images, plants, aur diseases lists
images = []
plants = []
diseases = []

# Directory traversal to collect images, plants, and diseases
for folder_name in os.listdir(image_directory):
    folder_path = os.path.join(image_directory, folder_name)
    if os.path.isdir(folder_path):
        parts = folder_name.split("__")
        plant_name, disease_name = parts
        for image_name in os.listdir(folder_path):
            images.append(image_name)
            plants.append(plant_name)
            diseases.append(disease_name)

# DataFrame
df = pd.DataFrame({'Image': images, 'Plant': plants, 'Disease': diseases})
# DataFrame to CSV file
df.to_csv('plant_disease.csv', index=False)


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/plant-disease-classification-merged-dataset'

In [ ]:
# Import Data Science Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
import cv2

# Tensorflow Libraries
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental import preprocessing

# System libraries
from pathlib import Path
import os.path

# Metrics
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [ ]:
print(df.head())

In [ ]:
path = '/kaggle/input/plant-disease-classification-merged-dataset'

In [ ]:
lst = os.listdir(path)
lst.sort()
lst

In [ ]:
usedClass = [
 'Apple__black_rot',
 'Apple__healthy',
 'Apple__rust',
 'Apple__scab',
 'Chili__healthy',
 'Chili__leaf curl',
 'Chili__leaf spot',
 'Chili__whitefly',
 'Chili__yellowish',
 'Coffee__cercospora_leaf_spot',
 'Coffee__healthy',
 'Coffee__red_spider_mite',
 'Coffee__rust',
 'Corn__common_rust',
 'Corn__gray_leaf_spot',
 'Corn__healthy',
 'Corn__northern_leaf_blight',
 'Grape__black_measles',
 'Grape__black_rot',
 'Grape__healthy',
 'Grape__leaf_blight_(isariopsis_leaf_spot)',
#  'Pepper_bell__bacterial_spot',
#  'Pepper_bell__healthy',
#  'Potato__early_blight',
#  'Potato__healthy',
#  'Potato__late_blight',
 'Rice__brown_spot',
 'Rice__healthy',
 'Rice__hispa',
 'Rice__leaf_blast',
 'Rice__neck_blast',
#  'Soybean__bacterial_blight',
#  'Soybean__caterpillar',
#  'Soybean__diabrotica_speciosa',
#  'Soybean__downy_mildew',
#  'Soybean__healthy',
#  'Soybean__mosaic_virus',
#  'Soybean__powdery_mildew',
#  'Soybean__rust',
#  'Soybean__southern_blight',
#  'Tea__algal_leaf',
#  'Tea__anthracnose',
#  'Tea__bird_eye_spot',
#  'Tea__brown_blight',
#  'Tea__healthy',
#  'Tea__red_leaf_spot',
 'Tomato__bacterial_spot',
 'Tomato__early_blight',
 'Tomato__healthy',
 'Tomato__late_blight',
 'Tomato__leaf_mold',
 'Tomato__mosaic_virus',
 'Tomato__septoria_leaf_spot',
 'Tomato__spider_mites_(two_spotted_spider_mite)',
 'Tomato__target_spot',
 'Tomato__yellow_leaf_curl_virus'
]

In [ ]:
image_dir = Path(path)

# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.PNG')) + list(image_dir.glob(r'**/*.png'))

labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)

In [ ]:
print(image_df.Filepath[0])

In [ ]:
labels_to_filter = usedClass
image_df = image_df[image_df['Label'].isin(labels_to_filter)]

In [ ]:
image_df

In [ ]:
image_df = image_df.reset_index()

In [ ]:
image_df

In [ ]:
# Display 16 picture of the dataset with their labels
random_index = np.random.randint(0, len(image_df), 16)
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(10, 10),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(image_df.Filepath[random_index[i]]))
    ax.set_title(image_df.Label[random_index[i]])
plt.tight_layout()
plt.show()

In [ ]:
train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
# train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=1./255, 
#     rotation_range = 40,
#     width_shift_range = 0.2,
#     height_shift_range = 0.2,
#     shear_range = 0.2,
#     zoom_range = 0.2,
#     horizontal_flip=True,
#     vertical_flip=True,
#     validation_split=0.2)

# test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=1./255)

In [ ]:
#our code
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    dtype = 'float32', preprocessing_function=tf.keras.applications.resnet.preprocess_input,validation_split=0.2)

# val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=1./255, 
#     rotation_range = 40,
#     width_shift_range = 0.2,
#     height_shift_range = 0.2,
#     shear_range = 0.2,
#     zoom_range = 0.2,
#     horizontal_flip=True,
#     vertical_flip=True,
#     validation_split=0.2)



test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    dtype = 'float32', preprocessing_function=tf.keras.applications.resnet.preprocess_input)

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 32

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# Train_Datagen = ImageDataGenerator(dtype = 'float32', preprocessing_function=tf.keras.applications.resnet.preprocess_input)
# Val_Datagen = ImageDataGenerator(dtype = 'float32', preprocessing_function=tf.keras.applications.resnet.preprocess_input)

In [ ]:
image_batch, label_batch = next(val_images)
image_batch.shape, label_batch.shape

In [ ]:
# print(image_batch)

In [ ]:
# print(label_batch)

In [ ]:
def create_pre_trained_model():
  """
  Initializes an ResNet50 model.
  
  Args:
    local_weights_file (string): path pointing to a pretrained weights H5 file
    
  Returns:
    pre_trained_model: the initialized InceptionV3 model
  """

  pre_trained_model = tf.keras.applications.ResNet50(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
) 

  

  # Make all the layers in the pre-trained model non-trainable
  for layer in pre_trained_model.layers:
    layer.trainable = False



  return pre_trained_model

In [ ]:
pre_trained_model = create_pre_trained_model()

In [ ]:
# Create checkpoint callback
checkpoint_path = "disease_classification_model_checkpoint"
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      save_weights_only=True,
                                      monitor="val_accuracy",
                                      save_best_only=True)

In [ ]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               patience = 5,
                               restore_best_weights = True) # if val loss decreases for 3 epochs in a row, stop training

In [ ]:
def output_of_last_layer(pre_trained_model):
  """
  Gets the last layer output of a model
  
  Args:
    pre_trained_model (tf.keras Model): model to get the last layer output from
    
  Returns:
    last_output: output of the model's last layer 
  """

  last_desired_layer = pre_trained_model.get_layer('conv5_block3_out')
#   print('last layer output shape: ', last_desired_layer.output_shape)
  last_output = last_desired_layer.output
#   print('last layer output: ', last_output)


  return last_output

In [ ]:
last_output = output_of_last_layer(pre_trained_model)

In [ ]:
print(last_output.shape)

In [ ]:
def create_final_model(pre_trained_model, last_output):
  """
  Appends a custom model to a pre-trained model
  
  Args:
    pre_trained_model (tf.keras Model): model that will accept the train/test inputs
    last_output (tensor): last layer output of the pre-trained model
    
  Returns:
    model: the combined model
  """
  x = layers.GlobalAveragePooling2D()(last_output)
  
  x =tf.keras.layers.Dense(units=512, activation='relu')(x)
  x =tf.keras.layers.Dropout(0.2)(x)
  x =tf.keras.layers.Dense(units=256, activation='relu')(x)
  x =tf.keras.layers.Dropout(0.2)(x)
  x =tf.keras.layers.Dense(units=128, activation='relu')(x)
  x =tf.keras.layers.Dense(units=36, activation='softmax')(x)      

  # Create the complete model by using the Model class
  model = Model(inputs=pre_trained_model.input, outputs=x)

  # Compile the model
  model.compile(optimizer = 'adam', 
                loss ='categorical_crossentropy',
                metrics = ['accuracy'])

  
  return model

In [ ]:
model = create_final_model(pre_trained_model, last_output)
total_params = model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

In [ ]:
model.summary()

In [ ]:
print('Number of trainable weights = {}'.format(len(model.trainable_weights)))

In [ ]:
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=5,
    callbacks=[
        early_stopping,
        checkpoint_callback,
    ]
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    dtype = 'float32', preprocessing_function=tf.keras.applications.resnet.preprocess_input)

In [ ]:
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    shuffle=False
)

In [ ]:
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
results = model.predict(test_images, verbose=0)

In [ ]:
print(results.shape)

In [ ]:
# from tensorflow.keras.applications.resnet50 import decode_predictions

In [ ]:
# decoded_predictions = decode_predictions(results, top=5)[0]
# for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
#     print(f"{i + 1}: {label} ({score:.2f})")

In [ ]:
# Display 16 picture of the dataset with their labels
# random_index = np.random.randint(0, len(image_df), 16)
# fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 10),
#                         subplot_kw={'xticks': [], 'yticks': []})

# for i, ax in enumerate(axes.flat):
#     ax.imshow(plt.imread(image_df.Filepath[random_index[i]]))
#     ax.set_title(image_df.Label[random_index[i]])
# plt.tight_layout()
# plt.show()

In [ ]:
# image_data = plt.imread(image_df.Filepath[1])
# print(image_data.shape)

# plt.imshow(image_data, cmap='gray')  # Use 'gray' colormap for grayscale images
# plt.title('Sample Image')  # Set a title for the plot (optional)
# plt.axis('off')  # Turn off axis labels and ticks (optional)
# plt.show()  # Show the plot





In [ ]:
# predictions = model.predict(image_data)

In [ ]:
# !pip install opencv-python

In [ ]:
# import cv2

# # Read an image file
# image = cv2.imread(image_df.Filepath[0])

# # Resize the image to a specific size (e.g., 300x200 pixels)
# new_size = (224, 224)
# resized_image = cv2.resize(image, new_size)

# # Save or display the resized image
# cv2.imwrite('output_image.jpg', resized_image)  # Save the resized image
# cv2.imshow('Resized Image', resized_image)  # Display the resized image
# cv2.waitKey(0)
# cv2.destroyAllWindows()